In [2]:
import pandas as pd 
import matplotlib.pyplot as plt 
from statsmodels.graphics.tsaplots import month_plot, \
    quarter_plot, plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error, \
    mean_squared_error, mean_absolute_percentage_error
import numpy as np
from sklearn.model_selection import ParameterGrid, GridSearchCV

from darts.timeseries import TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel

In [10]:
df = pd.read_csv('Hourly-train.csv', header=None).values.squeeze()\
    [1:, 1:].astype(np.float64).T
df

array([[ 605., 3124., 1828., ...,   19.,   21.,   15.],
       [ 586., 2990., 1806., ...,   30.,   16.,   13.],
       [ 586., 2862., 1897., ...,   12.,   17.,   13.],
       ...,
       [  nan,   nan,   nan, ...,   73.,   53.,   35.],
       [  nan,   nan,   nan, ...,   39.,   38.,   26.],
       [  nan,   nan,   nan, ...,   36.,   46.,   17.]])

In [16]:
df = df[~np.any(np.isnan(df), axis=1)]
df = pd.DataFrame(df)
df

,0,1,2,3,4,5,6,7,8,9,...,404,405,406,407,408,409,410,411,412,413
0,605.0,3124.0,1828.0,6454.0,4263.0,5780.0,52817.0,32479.0,39455.0,513.0,...,68.0,21.0,214.0,19.0,31.0,153.0,24.0,19.0,21.0,15.0
1,586.0,2990.0,1806.0,6324.0,4297.0,5497.0,51773.0,31018.0,38279.0,512.0,...,48.0,15.0,238.0,31.0,39.0,196.0,30.0,30.0,16.0,13.0
2,586.0,2862.0,1897.0,6075.0,4236.0,5306.0,50907.0,30033.0,36944.0,506.0,...,33.0,15.0,130.0,25.0,22.0,163.0,22.0,12.0,17.0,13.0
3,559.0,2809.0,1750.0,5949.0,4080.0,5178.0,50224.0,27716.0,35984.0,500.0,...,21.0,17.0,69.0,23.0,17.0,131.0,14.0,16.0,19.0,16.0
4,511.0,2544.0,1679.0,5858.0,3883.0,5235.0,49496.0,25067.0,33721.0,490.0,...,30.0,15.0,68.0,14.0,13.0,82.0,19.0,13.0,16.0,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,790.0,3705.0,1517.0,5563.0,4205.0,6077.0,51267.0,39019.0,42470.0,476.0,...,44.0,44.0,137.0,53.0,27.0,57.0,52.0,29.0,20.0,17.0
696,784.0,3652.0,1570.0,5739.0,4303.0,5644.0,52511.0,37450.0,41708.0,483.0,...,41.0,36.0,84.0,65.0,26.0,40.0,22.0,18.0,23.0,17.0
697,752.0,3460.0,1581.0,5760.0,4373.0,5380.0,53168.0,36211.0,40615.0,486.0,...,23.0,25.0,40.0,39.0,22.0,42.0,16.0,18.0,17.0,13.0
698,739.0,3248.0,1570.0,5562.0,4270.0,5174.0,52593.0,35020.0,39152.0,482.0,...,15.0,15.0,28.0,40.0,14.0,29.0,18.0,13.0,16.0,17.0
